# Prepare dataset

In [1]:
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import torch
import torch.nn as nn

# local path
# FAULT_DATA_DIR = "D:\\Windows_Storage\\Storage\\Github\\phm_etching_01M01-02\\M02_Groups_same_length\\fault"
# HEALTHY_DATA_DIR = "D:\\Windows_Storage\\Storage\\Github\\phm_etching_01M01-02\\M02_Groups_same_length\\healthy"

# server path
FAULT_DATA_DIR = "C:\\Users\\User\\Desktop\\Ricardo\\phm_etching_01M01-02\\M02_Groups_recipe_67_same_length\\fault\\"
HEALTHY_DATA_DIR = "C:\\Users\\User\Desktop\\Ricardo\\phm_etching_01M01-02\\M02_Groups_recipe_67_same_length\\healthy\\"
HEALTHY_DATA_NUM = 51
MODEL_SAVE_PATH = "C:\\Users\\User\Desktop\\Ricardo\\phm_etching_01M01-02\\Models\\210619_LSTM"

In [2]:
os.chdir(FAULT_DATA_DIR)

# read in fault data
for fileIndex, file in enumerate(tqdm(os.listdir())):
    if fileIndex == 0:
        data_all = pd.read_csv(file, encoding="utf8")
    else:
        data_import = pd.read_csv(file, encoding="utf8")
        data_all = data_all.append(data_import, ignore_index = True)
        
os.chdir(HEALTHY_DATA_DIR)

healthy_file_list = os.listdir()
np.random.shuffle(healthy_file_list)

for fileIndex, file in enumerate(tqdm(healthy_file_list[:HEALTHY_DATA_NUM])):
    data_import = pd.read_csv(file, encoding="utf8")
    data_all = data_all.append(data_import, ignore_index = True)

100%|██████████| 51/51 [00:01<00:00, 43.50it/s]


In [3]:
# configurations
SEQ_LENGTH = 500

In [4]:
# create datasets with given sequence length
def sliding_windows(data, seq_length, selected_column):
    x = []
    y = []
    
    # calculate max min for each column
    data_temp = data.drop(columns=['time', 'Tool', 'Lot', 'Lot-runnum'])
    
    column_list = data_temp.columns.tolist()
    max_value_list = data_temp.max().tolist()
    min_value_list = data_temp.min().tolist()
    
    # normalize the data
    for column in data.columns:
        if column in ['time', 'Tool', 'Lot', 'Lot-runnum']:
            continue
            
        # x ′ = ( x − x m i n ) / ( x m a x − x m i n )
        column_index = column_list.index(column)
        data[column] = data[column].apply(lambda x: \
                                          (x - min_value_list[column_index]) / (max_value_list[column_index] - \
                                                                   min_value_list[column_index]))
        
#     print(data.head(5))

    for i in tqdm(range(len(data)-seq_length-1)):
        # create x and y 
        data_x = data.iloc[i:(i+seq_length), :]
        data_y = data.loc[[(i+seq_length)], [selected_column]]
        
        # skip if contains for than two Lot-runnum and np.nan appears in y
        if data_x["Lot-runnum"].nunique() > 1:
            continue
            
        # drop nan value
        if data_y[selected_column].isnull().values.any() or\
        data_x.isnull().values.any() :
            continue
        
        # drop unneccessary columns
        data_x.drop(columns=['time', 'Tool', 'Lot', 'TTF_FlowCool Pressure Dropped Below Limit',
       'TTF_Flowcool Pressure Too High Check Flowcool Pump', 'TTF_Flowcool leak', 'Lot-runnum'],
                     inplace=True)
        
        x.append(data_x)
        y.append(data_y)
        

    return np.array(x),np.array(y)

# Pytorch Reference
#### CUDA Implementation Reference
* https://pytorch.org/docs/stable/generated/torch.Tensor.to.html
* https://pytorch.org/docs/stable/tensors.html
* https://pytorch.org/docs/stable/generated/torch.Tensor.cuda.html

#### Saving & Load Model Reference:
* https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [5]:
# activate cuda

# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()
print(is_cuda)

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

True


In [6]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = SEQ_LENGTH
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size).to(device))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size).to(device))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

# TTF_FlowCool Pressure Dropped Below Limit

In [7]:
SELECTED_COLUMN = 'TTF_FlowCool Pressure Dropped Below Limit'
x, y = sliding_windows(data_all, SEQ_LENGTH, SELECTED_COLUMN)
y = np.squeeze(y, axis=2)

print(x.shape, y.shape)

  0%|          | 0/102539 [00:00<?, ?it/s]c:\users\user\anaconda3\envs\nlp\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 102539/102539 [02:25<00:00, 705.52it/s]


(47673, 500, 21) (47673, 1)


In [8]:
train_size = int(len(y) * 0.67)
test_size = len(y) - train_size


if is_cuda:
    dataX = Variable(torch.Tensor(np.array(x)).cuda(device=0))
    dataY = Variable(torch.Tensor(np.array(y)).cuda(device=0))

    trainX = Variable(torch.Tensor(np.array(x[0:train_size])).cuda(device=0))
    trainY = Variable(torch.Tensor(np.array(y[0:train_size])).cuda(device=0))

    testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])).cuda(device=0))
    testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])).cuda(device=0))
else:
    dataX = Variable(torch.Tensor(np.array(x)))
    dataY = Variable(torch.Tensor(np.array(y)))

    trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
    trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

    testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
    testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

print(trainX.shape, trainY.shape, "\n", testX.shape, testY.shape)

torch.Size([31940, 500, 21]) torch.Size([31940, 1]) 
 torch.Size([15733, 500, 21]) torch.Size([15733, 1])


In [9]:
num_epochs = 2000
learning_rate = 0.01

input_size = 21
hidden_size = 5
num_layers = 1

num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

if is_cuda:
    lstm = lstm.cuda()

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
#     print(outputs)
#     print(trainY)
    
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
        
# save model
torch.save(lstm.state_dict(), MODEL_SAVE_PATH)

RuntimeError: CUDA out of memory. Tried to allocate 1.26 GiB (GPU 0; 11.00 GiB total capacity; 8.61 GiB already allocated; 0 bytes free; 9.28 GiB reserved in total by PyTorch)

In [ ]:
# prediction
lstm.eval()
train_predict = lstm(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

# data_predict = sc.inverse_transform(data_predict)
# dataY_plot = sc.inverse_transform(dataY_plot)

plt.figure(figsize=(40,10))

plt.axvline(x=train_size, c='r', linestyle='--')

plt.plot(dataY_plot, color="blue", linewidth=2)
plt.plot(data_predict, color="red", linewidth=2)
plt.suptitle('Time-Series Prediction')
plt.savefig("C:/Users/User/Desktop/Ricardo/PHM_ion-mill-etch-tool/Graphs/TTF_FlowCoolPressureDroppedBelowLimit.svg", format='svg', bbox_inches="tight")
plt.show()